In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import sys
import numpy as np

In [2]:
dbpedia_endpoint = "https://dbpedia.org/sparql"
sparql = SPARQLWrapper(dbpedia_endpoint, agent='sparqlwrapper 1.8.6.dev0 (rdflib.github.io/sparqlwrapper)')

In [3]:
class_qry1 = """PREFIX wikiID: <http://www.wikidata.org/entity/>
select distinct ?ent ?type where { 
?ent owl:sameAs wikiID:"""
#Q76 to come here in between
class_qry2 = """ . ?ent rdf:type ?type 
FILTER( regex( ?type, "http://dbpedia.org/ontology/"))
}"""

In [4]:
#function to fetch results of search term
def fetch_ent_typ(qry):
       dbpedia_catg = ''
       lst_dbpd_typs = []
       try:
          sparql.setQuery(qry)
          sparql.setReturnFormat(JSON)
          results = sparql.query().convert()    #results['boolean']
          #print(results)
          if not len(results['results']['bindings']) == 0:
             for item in results['results']['bindings']:                   
                try:
                   dbpedia_catg = item['type']['type']
                   #print(dbpedia_catg)
                   lst_dbpd_typs.append('dbo:'+str(item['type']['value'].split('/')[-1]))
                except Exception as ex:
                   print(ex)
                   pass
       except Exception as ex:
          print('exception came: ',ex)
          pass
       if dbpedia_catg == 'uri':
          dbpedia_catg = 'resource'
       return dbpedia_catg, lst_dbpd_typs

In [7]:
#for i in range(4):
i=3
if i == 3:
    inp_file = '../data_files/simpques_wiki_smart/outdata'+str(i+1)+'.json'
    out_file = '../data_files/simpques_wiki_smart/output_sq_res'+str(i+1)+'.json'
    df_sq = pd.read_json(inp_file)
    #print(len(df_sq))
    lst_id, lst_ques, lst_datatype, lst_res = [], [], [], []
    for i in range(len(df_sq)):
        lst_id.append(df_sq['simple_questions_wikidata'][i]['id'])
        lst_ques.append(df_sq['simple_questions_wikidata'][i]['question'])
        wiki_obj = df_sq['simple_questions_wikidata'][i]['object']
        qry = class_qry1 + wiki_obj + class_qry2
        #print(qry)
        dbpedia_catg, dbpedia_type = fetch_ent_typ(qry)  #should return a category and a list 
        #print(dbpedia_catg, dbpedia_type)
        lst_datatype.append(dbpedia_catg)
        lst_res.append(dbpedia_type)
    pdf_data_v = {'id': lst_id, 'question': lst_ques, 'category': lst_datatype, 'type': lst_res}
    new_df = pd.DataFrame(pdf_data_v, columns = ['id','question','category','type'])
    new_df.to_json(out_file, orient='records', indent=len(new_df.columns))

## Processing SmartTask datasets
DBpedia datasets

In [10]:
dbpd_train_df = pd.read_json('../data_files/DBpedia/smarttask_dbpedia_train.json')
dbpd_test_df = pd.read_json('../data_files/DBpedia/smarttask_dbpedia_test_questions.json')
dbpd_test_gold_df = pd.read_json('../data_files/DBpedia/smarttask_dbpedia_test.json')
print(len(dbpd_train_df))
print(len(dbpd_test_df))
print(len(dbpd_test_gold_df))

17571
4381
4381


In [14]:
#merging train and test files
final_dbpd_df = pd.concat([dbpd_train_df, dbpd_test_df])
print(len(final_dbpd_df))
#fetching all records of web questions
webques_dbpd_inp_file = '../data_files/webques_wiki_smart/outdata_webques_dbepdia.json' 
wbqsp_dbpd_df = pd.read_json(webques_dbpd_inp_file)

21952


In [15]:
print('len of webqsp original: ',len(wbqsp_dbpd_df))
wbqsp_dbpd_df = wbqsp_dbpd_df.loc[wbqsp_dbpd_df['category']!='']
print('len of webqsp finetuned: ',len(wbqsp_dbpd_df))
final_dbpd_df = pd.concat([final_dbpd_df, wbqsp_dbpd_df])
for i in range(4):
    inp_file = '../data_files/simpques_wiki_smart/output_sq_res'+str(i+1)+'.json'  
    extra_dbpd_df = pd.read_json(inp_file)
    print('tot length of extra df: ',len(extra_dbpd_df))
    not_null_df = extra_dbpd_df.loc[extra_dbpd_df['category']!='']
    print('tot length of refined extra df: ',len(not_null_df))
    final_dbpd_df = pd.concat([final_dbpd_df, not_null_df])
print(len(final_dbpd_df))
#creating new id as old id no more holds good after merging all
final_dbpd_df['new_id'] = [x for x in range(len(final_dbpd_df))]
out_dbpd_file = '../data_files/simpques_wiki_smart/out_dbpd_train_test_merge.json'
final_dbpd_df.to_json(out_dbpd_file, orient='records', indent=len(final_dbpd_df.columns))
final_dbpd_df.tail(6)

len of webqsp original:  4131
len of webqsp finetuned:  4131
tot length of extra df:  5622
tot length of refined extra df:  5031
tot length of extra df:  6000
tot length of refined extra df:  5137
tot length of extra df:  13482
tot length of refined extra df:  11945
tot length of extra df:  2821
tot length of refined extra df:  2518
50714


,id,question,category,type,new_id
2813,2814,what kinds of music is played by season's end,resource,"[dbo:Genre, dbo:TopicalConcept, dbo:MusicGenre]",50708
2814,2815,which asteroid group is 6753 fursenko a member...,resource,[dbo:Album],50709
2815,2816,What language is azhakiya ravanan filmed in?,resource,[dbo:Language],50710
2816,2817,who developed outnumbered!,resource,"[dbo:Company, dbo:Organisation, dbo:Agent]",50711
2818,2819,which position did herby fortunat play in foot...,resource,[dbo:Person],50712
2820,2821,who is a person that was born in sao paulo,resource,"[dbo:Person, dbo:Artist, dbo:MusicalArtist, db...",50713


In [16]:
#renaming ids and dropping previous ids
final_dbpd_df.drop("id", axis=1, inplace=True)
final_dbpd_df.rename(columns={'new_id':'id'}, inplace=True)
final_dbpd_df = final_dbpd_df[['id','question','category','type']]

In [17]:
#splitting in train and test files
msk = np.random.rand(len(final_dbpd_df)) < 0.8
dbpd_train_df = final_dbpd_df[msk]
print(len(dbpd_train_df))
dbpd_test_gold_df = final_dbpd_df[~msk]
print(len(dbpd_test_gold_df))
out_dbpd_train_file = '../data_files/DBpedia/out_dbpd_train.json'
dbpd_train_df.to_json(out_dbpd_train_file, orient='records', indent=len(dbpd_train_df.columns))
out_dbpd_test_gold_file = '../data_files/DBpedia/out_dbpd_test_gold.json'
dbpd_test_gold_df.to_json(out_dbpd_test_gold_file, orient='records', indent=len(dbpd_test_gold_df.columns))
dbpd_test_df = dbpd_test_gold_df[['id','question']]
out_dbpd_test_file = '../data_files/DBpedia/out_dbpd_test.json'
dbpd_test_df.to_json(out_dbpd_test_file, orient='records', indent=len(dbpd_test_df.columns))

40621
10093


## Processing SmartTask datasets
Wikidata datasets

In [101]:
wikidata_endpoint = "https://query.wikidata.org/sparql"
sparql_wiki = SPARQLWrapper(wikidata_endpoint, agent='sparqlwrapper 1.8.6.dev0 (rdflib.github.io/sparqlwrapper)')

In [102]:
wiki_train_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_train.json')
wiki_test_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_test_gold.json')
wiki_test_gold_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_test.json')
wiki_labels_df = pd.read_csv('../data_files/Wikidata/wikidata_qid_label.csv',sep=',')
wiki_labels_df = wiki_labels_df[['qid','label']]
print(len(wiki_train_df))
print(len(wiki_test_df))
print(len(wiki_test_gold_df))
print(len(wiki_labels_df))
wiki_labels_df

18251
4571
4571
2644


,qid,label
0,Q184485,performing arts
1,Q30168244,human hybrids in a work of fiction
2,Q2369882,funny animal
3,Q904092,homopolysaccharide
4,Q4373292,physical property
...,...,...
2639,Q10553309,legislative house
2640,Q1307779,integrated municipality
2641,Q489983,Reserve Officers' Training Corps
2642,Q849275,civil time


In [103]:
#merging train and test files
final_wiki_df = pd.concat([wiki_train_df, wiki_test_df])
print(len(final_wiki_df))
final_wiki_df.head(2)

22822


,id,question,category,type
0,19719,What periodical literature does Delta Air Line...,resource,"[publication, recurring, intellectual work, te..."
1,15554,Who is the child of Ranavalona I's husband?,resource,"[person, omnivore, natural person]"


In [104]:
qry_pt1 = 'select distinct ?label where { wd:'
qry_pt2 = ' rdfs:label ?label . FILTER (langMatches( lang(?label), "EN" ) ) } LIMIT 1'

In [105]:
#function to fetch answer from wikidata endpoint
def fetch_label(qry):
    res = ''
    sparql_wiki.setQuery(qry)
    sparql_wiki.setReturnFormat(JSON)
    results = sparql_wiki.query().convert() 
    for item in results['results']['bindings']: 
        res = item['label']['value'].lower().strip()
    return res

In [110]:
#fetching all records of web questions and simp ques
def concat_df(folder_name,num_files,file_name,subject,final_wiki_df,local_qid_lbl_df):
    for j in range(num_files):
        webques_wiki_inp_file = folder_name+file_name+str(j+1)+'.json' 
        webques_wiki_out_file = folder_name+'lblOutput_'+file_name+str(j+1)+'.json' 
        wbqsp_wiki_df = pd.read_json(webques_wiki_inp_file)
        lst_id, lst_ques, lst_datatype, lst_res, lst_qid_lbl = [], [], [], [], []
        wiki_catg = ''
        for i in range(len(wbqsp_wiki_df)):
            #print(wbqsp_wiki_df[subject][i]['datatype']['results']['bindings'])
            #break
            lst_id.append(wbqsp_wiki_df[subject][i]['id']) 
            lst_ques.append(wbqsp_wiki_df[subject][i]['question'])
            try:
               results = wbqsp_wiki_df[subject][i]['datatype']
               lst_tmp_lbls = []
               lst_tmp_qid_lbl = []
               if results == 'date':
                    lst_datatype.append('literal')
                    lst_res.append(['date'])
                    lst_qid_lbl.append(lst_tmp_qid_lbl)
               elif len(results['results']['bindings']) != 0:
                    for item in results['results']['bindings']:   
                       qid = str(item['type']['value'].split('/')[-1])
                       filtered_local_qid_lbl_df = local_qid_lbl_df.loc[local_qid_lbl_df['qid']==qid]
                       if filtered_local_qid_lbl_df.empty: 
                          try:
                             wiki_catg = item['type']['type']
                             qry = qry_pt1 + qid + qry_pt2
                             temp_ans_label = fetch_label(qry)
                             if not temp_ans_label in lst_tmp_lbls:
                                lst_tmp_lbls.append(temp_ans_label) 
                                lst_tmp_qid_lbl.append(qid+'-'+temp_ans_label)
                          except Exception as ex:
                             wiki_catg = ''
                             lst_tmp_lbls.append(lst_tmp_lbls)
                             print(ex)
                             pass   
                       else:
                          wiki_catg = 'resource'
                          loc_val = filtered_local_qid_lbl_df['label'].values[0]
                          if not loc_val in lst_tmp_lbls:
                             lst_tmp_lbls.append(loc_val)
                             lst_tmp_qid_lbl.append(qid+'-'+loc_val)
                    if wiki_catg == 'uri':
                       wiki_catg = 'resource'
                    lst_datatype.append(wiki_catg)
                    lst_res.append(lst_tmp_lbls)  
                    lst_qid_lbl.append(lst_tmp_qid_lbl)
               else:
                    lst_datatype.append('')
                    lst_res.append(lst_tmp_lbls) 
                    lst_qid_lbl.append(lst_tmp_qid_lbl)
            except Exception as ex:
                if wiki_catg == 'uri':
                   wiki_catg = 'resource'
                lst_datatype.append('')
                lst_res.append(lst_tmp_lbls)
                lst_qid_lbl.append(lst_tmp_qid_lbl)
                #print(ex)
            if i % 2000 == 0 and i!=0:
               print(i)
        pdf_data_v = {'id': lst_id, 'question': lst_ques, 'category': lst_datatype, 'type': lst_res, 'wiki_id_label': lst_qid_lbl}
        new_df = pd.DataFrame(pdf_data_v, columns = ['id','question','category','type','wiki_id_label'])
        new_df.to_json(webques_wiki_out_file, orient='records', indent=len(new_df.columns))
        print('orig len',len(new_df))
        new_df = new_df.loc[new_df['category']!='']
        print('reduced len',len(new_df))
        new_df = new_df.loc[np.array(list(map(len,new_df.type.values)))>0]
        new_df = new_df[['id','question','category','type']]
        final_wiki_df = pd.concat([final_wiki_df, new_df])
        print('j is:',j)
    return final_wiki_df

In [111]:
folder_name = '../data_files/webques_wiki_smart/'
file_name = 'outdata_wq'
subject = 'web_questions_wikidata'
new_final_wiki_df = concat_df(folder_name,3,file_name,subject,final_wiki_df,wiki_labels_df)
print(len(new_final_wiki_df)) 
new_final_wiki_df.to_json('../data_files/Wikidata/final_wiki_data_1.json',orient='records', indent=len(new_final_wiki_df.columns))

2000
orig len 3072
reduced len 2699
j is: 0
orig len 16
reduced len 11
j is: 1
orig len 1628
reduced len 1421
j is: 2
26953


In [112]:
# doing same for simple questions
folder_name = '../data_files/simpques_wiki_smart/'
file_name = 'outdata'
subject = 'simple_questions_wikidata'
wqsp_sq_final_wiki_df = concat_df(folder_name,4,file_name,subject,new_final_wiki_df,wiki_labels_df)
print(len(wqsp_sq_final_wiki_df)) 
wqsp_sq_final_wiki_df.to_json('../data_files/Wikidata/final_wiki_data_2.json', orient='records', indent=len(wqsp_sq_final_wiki_df.columns))

2000
4000
orig len 5622
reduced len 5541
j is: 0
2000
4000
orig len 6000
reduced len 5898
j is: 1
2000
4000
6000
8000
10000
12000
orig len 13482
reduced len 13292
j is: 2
2000
orig len 2821
reduced len 2784
j is: 3
54468


In [113]:
#creating new id as old id no more holds good after merging all
wqsp_sq_final_wiki_df['new_id'] = [x for x in range(len(wqsp_sq_final_wiki_df))]
out_wiki_file = '../data_files/out_wiki_train_test_merge.json'
wqsp_sq_final_wiki_df.to_json(out_wiki_file, orient='records', indent=len(wqsp_sq_final_wiki_df.columns))
wqsp_sq_final_wiki_df.tail(6)

,id,question,category,type,new_id
2815,2816,What language is azhakiya ravanan filmed in?,resource,"[human language, languoid]",54462
2816,2817,who developed outnumbered!,resource,[software company],54463
2817,2818,what is the sex of seumas o'kelly?,resource,"[gender, identity, sex]",54464
2818,2819,which position did herby fortunat play in foot...,resource,[position],54465
2819,2820,What is nikolaj frobenius's profession?,resource,"[job, specialty, occupation]",54466
2820,2821,who is a person that was born in sao paulo,resource,"[natural person, omnivore, person]",54467


In [114]:
#renaming ids and dropping previous ids
wqsp_sq_final_wiki_df.drop("id", axis=1, inplace=True)
wqsp_sq_final_wiki_df.rename(columns={'new_id':'id'}, inplace=True)
wqsp_sq_final_wiki_df = wqsp_sq_final_wiki_df[['id','question','category','type']]

In [115]:
#splitting in train and test files
msk = np.random.rand(len(wqsp_sq_final_wiki_df)) < 0.8
wiki_train_df = wqsp_sq_final_wiki_df[msk]
print(len(wiki_train_df))
wiki_test_gold_df = wqsp_sq_final_wiki_df[~msk]
print(len(wiki_test_gold_df))
out_wiki_train_file = '../data_files/Wikidata/out_wiki_train.json'
wiki_train_df.to_json(out_wiki_train_file, orient='records', indent=len(wiki_train_df.columns))
out_wiki_test_gold_file = '../data_files/Wikidata/out_wiki_test_gold.json'
wiki_test_gold_df.to_json(out_wiki_test_gold_file, orient='records', indent=len(wiki_test_gold_df.columns))
wiki_test_df = wiki_test_gold_df[['id','question']]
out_wiki_test_file = '../data_files/Wikidata/out_wiki_test.json'
wiki_test_df.to_json(out_wiki_test_file, orient='records', indent=len(wiki_test_df.columns))

43604
10864


In [116]:
wiki_train_df

,id,question,category,type
0,0,What periodical literature does Delta Air Line...,resource,"[publication, recurring, intellectual work, te..."
1,1,Who is the child of Ranavalona I's husband?,resource,"[person, omnivore, natural person]"
2,2,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,[boolean]
3,3,Which is the operating income for Qantas?,literal,[number]
4,4,which cola starts with the letter p,resource,"[soft drink, trademark, carbonated beverage, n..."
...,...,...,...,...
2816,54463,who developed outnumbered!,resource,[software company]
2817,54464,what is the sex of seumas o'kelly?,resource,"[gender, identity, sex]"
2818,54465,which position did herby fortunat play in foot...,resource,[position]
2819,54466,What is nikolaj frobenius's profession?,resource,"[job, specialty, occupation]"


In [117]:
wiki_test_gold_df

,id,question,category,type
14,14,After what is Marathon named and what is the c...,resource,"[combat, omnivore, historical event, military ..."
23,23,How many dimensions have a Captain America?,literal,[number]
27,27,What country is the current leader of the Afri...,resource,"[state, political territorial entity, country]"
29,29,n/a,resource,[human settlement]
44,44,Is the life expectancy of Indonesia 55.3528?,boolean,[boolean]
...,...,...,...,...
2799,54446,who was the director for the film good girls g...,resource,"[natural person, omnivore, person]"
2800,54447,which rock group made all you can ever learn i...,resource,[musical ensemble]
2810,54457,what county is in lima?,resource,[county of the United States]
2812,54459,where is abby mann's nationality,resource,"[political territorial entity, federal system,..."


In [118]:
wiki_test_df

,id,question
14,14,After what is Marathon named and what is the c...
23,23,How many dimensions have a Captain America?
27,27,What country is the current leader of the Afri...
29,29,n/a
44,44,Is the life expectancy of Indonesia 55.3528?
...,...,...
2799,54446,who was the director for the film good girls g...
2800,54447,which rock group made all you can ever learn i...
2810,54457,what county is in lima?
2812,54459,where is abby mann's nationality
